# Frasier Dialog Analysis
-----
This project contains several functions that manipulate script data from every season of the hit NBC sitcom "Frasier".

In [1]:
import csv
import numpy as np
import re

from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
from bokeh.models import NumeralTickFormatter, LabelSet, Label, Legend, Title
from bokeh.layouts import gridplot

output_notebook(hide_banner=True)
filename = 'script.csv'
fileobj = open(filename, 'r')
readerobj = csv.reader(fileobj)
data = []
for r in readerobj:
    data.append(r)
fileobj.close()
temp = np.asarray(data)
script = temp[1:,:]

-----
## Functions

In [2]:
def simplify_dialog(text, brackets="()[]"):
    count = [0] * (len(brackets) // 2) # count open/close brackets
    saved_chars = []
    for character in text:
        for i, b in enumerate(brackets):
            if character == b: # found bracket
                kind, is_close = divmod(i, 2)
                count[kind] += (-1)**is_close # `+1`: open, `-1`: close
                if count[kind] < 0: # unbalanced bracket
                    count[kind] = 0  # keep it
                else:  # found bracket to remove
                    break
        else: # character is not a [balanced] bracket
            if not any(count): # outside brackets
                saved_chars.append(character)
    no_cues = "".join(saved_chars)
    no_chars = " ".join(re.findall("[a-zA-Z]+", no_cues))
    to_list = re.sub("[^\w]", " ",  no_chars).split()
    return to_list

def strip_season(col):
    season = re.sub("[/]", " ", col).split()
    return season[4]

def namesearch(db,name):
    condition = np.where(db == str(name))[0]
    newlist = db[condition,2]
    output = (str(name),np.size(newlist))
    return output

def seasonrename(num):
    season = re.sub("[_]", " ", num)
    upper = season.capitalize()
    return upper

def linesbyseason(db,num,name, printer = False):
    season = []
    for i in range(len(db)):
        s = strip_season(db[i,0])
        season.append(s)
    newarr = np.asarray(season)
    season_cond = np.where(newarr == str(num))[0]
    newlist = db[season_cond,1]
    name_cond = np.where(newlist == str(name))
    name_count = newlist[name_cond]
    percent = (np.size(name_count))/(np.size(newlist))
    convert = '{:.2%}'.format(percent)
    if printer == True:
        return(str(name)+" had "+str(np.size(name_count))+" lines in "
            +str(seasonrename(num))+" ("+str(convert)+" of the total lines)")
    else:
        return np.size(name_count)

def linesperseason(db,name):
    seasonlist = ('season_1','season_2','season_3','season_4','season_5','season_6',
    'season_7','season_8','season_9','season_10','season_11')
    s = []
    n = []
    for i in seasonlist:
        s.append(i)
        n.append(linesbyseason(db,i,str(name)))
    newlist = np.array(n)
    return newlist

-----
## Data Analysis

In [3]:
print(linesbyseason(script,'season_1','Frasier',printer=True))
print(linesbyseason(script,'season_1','Niles',printer=True))
print(linesbyseason(script,'season_1','Roz',printer=True))
print(linesbyseason(script,'season_1','Martin',printer=True))
print(linesbyseason(script,'season_1','Daphne',printer=True))

Frasier had 2192 lines in Season 1 (38.82% of the total lines)
Niles had 795 lines in Season 1 (14.08% of the total lines)
Roz had 312 lines in Season 1 (5.53% of the total lines)
Martin had 813 lines in Season 1 (14.40% of the total lines)
Daphne had 445 lines in Season 1 (7.88% of the total lines)


In [4]:
totalines = np.size(script[:,1])
ftotal = namesearch(script,'Frasier')
ntotal = namesearch(script,'Niles')
mtotal = namesearch(script,'Martin')
dtotal = namesearch(script,'Daphne')
rtotal = namesearch(script,'Roz')

totals = [ftotal,ntotal,mtotal,dtotal,rtotal]
net = 0
for i in range(len(totals)):
    character = totals[i][0]
    dec = (totals[i][1])/totalines
    percent = '{:.2%}'.format(dec)
    net += dec
    print(str(character)+" delivered "+str(percent)+" percent of the total lines in all 11 seasons")
remain = 1 - net
rpercent = '{:.2%}'.format(remain)
print(str(rpercent)+" of the total lines were delivered by other characters")

Frasier delivered 32.98% percent of the total lines in all 11 seasons
Niles delivered 15.92% percent of the total lines in all 11 seasons
Martin delivered 11.93% percent of the total lines in all 11 seasons
Daphne delivered 8.20% percent of the total lines in all 11 seasons
Roz delivered 7.40% percent of the total lines in all 11 seasons
23.57% of the total lines were delivered by other characters


In [5]:
lines_frasier = linesperseason(script,'Frasier')
lines_niles = linesperseason(script,'Niles')
lines_martin = linesperseason(script,'Martin')
lines_roz = linesperseason(script,'Roz')
lines_daphne = linesperseason(script,'Daphne')

In [6]:
output_file("character_dialog.html")

szns = [1,2,3,4,5,6,7,8,9,10,11]

p = figure(plot_width=700, plot_height=300,x_range=(1,11))
p.toolbar.logo = None
p.toolbar_location = None

f = p.line(szns,lines_frasier,color="firebrick",line_width=2)
n = p.line(szns,lines_niles,color="navy",line_width=2)
r = p.line(szns,lines_roz,color="orange",line_width=2)
m = p.line(szns,lines_martin,color="green",line_width=2)
d = p.line(szns,lines_daphne,color="magenta",line_width=2)

legend = Legend(items=[
    ("Frasier"   , [f]),
    ("Niles" , [n]),
    ("Roz" , [r]),
    ("Martin" , [m]),
    ("Daphne" , [d])
], location=(0, 95))

p.xaxis[0].axis_label = 'Season'
p.yaxis[0].axis_label = 'Lines of Dialog'
p.xaxis[0].formatter = NumeralTickFormatter(format="Season 0")
p.xaxis[0].ticker.desired_num_ticks = 12

p.add_layout(Title(text="Frasier Dialog by Character Lines", align="center"), "above")

p.add_layout(legend, 'right')

show(p)

----
Data retrieved from "The Frasier Archive" (kacl780.net), scraped and organized by Kaggle user sulabhbista